In [43]:
!pip install pandas numpy scikit-learn

In [44]:
import sys
print(sys.executable)
from collections import defaultdict


c:\Users\anahi\miniconda3\envs\dooleyhelps\python.exe


In [45]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import os, json, math
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import os
# --- Figure out project root and key dirs ---
cwd = os.getcwd()
if os.path.basename(cwd) == "Model":
    PROJECT_ROOT = os.path.dirname(cwd)
else:
    PROJECT_ROOT = cwd  # if you're already at the root

MODEL_DIR = os.path.join(PROJECT_ROOT, "Model")
OUT_DIR   = os.path.join(PROJECT_ROOT, "out")

print("CWD:", cwd)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("MODEL_DIR:", MODEL_DIR)
print("OUT_DIR:", OUT_DIR)

# Make sure Python can import from Model/
if MODEL_DIR not in sys.path:
    sys.path.append(MODEL_DIR)

from track_graduation import track_grad

# --- Paths to your data files ---
SYNTHETIC_COURSES_PATH = os.path.join(MODEL_DIR, "synthetic_courses.json")
SYNTHETIC_PREF_PATH    = os.path.join(MODEL_DIR, "synthetic_pref.json")

print("SYNTHETIC_COURSES_PATH:", SYNTHETIC_COURSES_PATH)
print("SYNTHETIC_PREF_PATH:", SYNTHETIC_PREF_PATH)

CWD: c:\Users\anahi\OneDrive\Documents\EMORY\Fall 2025\CS 370\Model
PROJECT_ROOT: c:\Users\anahi\OneDrive\Documents\EMORY\Fall 2025\CS 370
MODEL_DIR: c:\Users\anahi\OneDrive\Documents\EMORY\Fall 2025\CS 370\Model
OUT_DIR: c:\Users\anahi\OneDrive\Documents\EMORY\Fall 2025\CS 370\out
SYNTHETIC_COURSES_PATH: c:\Users\anahi\OneDrive\Documents\EMORY\Fall 2025\CS 370\Model\synthetic_courses.json
SYNTHETIC_PREF_PATH: c:\Users\anahi\OneDrive\Documents\EMORY\Fall 2025\CS 370\Model\synthetic_pref.json


In [46]:
from Courses_Qualified_new import build_qualified_courses_for_student

# Load synthetic user data (still needed for build_student_context, etc.)
with open(SYNTHETIC_COURSES_PATH, "r") as f:
    synthetic_courses = {rec["shared_id"]: rec for rec in json.load(f)}

with open(SYNTHETIC_PREF_PATH, "r") as f:
    synthetic_prefs = {rec["shared_id"]: rec for rec in json.load(f)}

# 🔹 Build per-student qualified courses using Mongo catalog + synthetic user
shared_id = "000025"  # or whichever synthetic student you're testing
candidate_courses = build_qualified_courses_for_student(shared_id)

print("students:", len(synthetic_courses), "candidate courses:", len(candidate_courses))


[INFO] Loaded 2094 docs from DetailedCourses.CoursesEnriched.
[INFO] For student 000025, 1419 courses qualified.
[INFO] Saved → out\courses_qualified_000025.json
students: 40 candidate courses: 1419


In [47]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

# --- Connect to MongoDB and CoursesEnriched for GER tags ---
load_dotenv()
uri = os.getenv("DB_URI")
client = MongoClient(uri)

detailed_db = client["DetailedCourses"]
enriched_col = detailed_db["CoursesEnriched"]

def build_ger_map():
    ger_map = {}
    cursor = enriched_col.find({}, {"_id": 0, "code": 1, "ger": 1})
    for doc in cursor:
        key = doc["code"]
        ger_map[key] = doc.get("ger", [])
    return ger_map


GER_MAP = build_ger_map()
print("Loaded GER_MAP with", len(GER_MAP), "courses")


Loaded GER_MAP with 1049 courses


In [48]:
from pymongo import MongoClient
import os

def load_ger_mapping_from_mongo():
    """
    Build a mapping: code -> [GER tags] from MongoDB.
    Assumes 'ger' is already stored as an array in the DB.
    """
    # Use whatever env var you already use (DB_URI / MONGODB_URI / ATLAS_URI, etc.)
    uri = os.environ.get("MONGODB_URI") or os.environ.get("DB_URI")
    if uri is None:
        raise RuntimeError("No Mongo URI found in environment (MONGODB_URI / DB_URI).")

    client = MongoClient(uri)

    # Adjust database / collection names if yours are different
    detailed = client["DetailedCourses"]["DetailedCourses"]

    ger_by_code = {}
    for doc in detailed.find({}, {"_id": 0, "code": 1, "ger": 1}):
        code = doc.get("code")
        if not code:
            continue
        ger = doc.get("ger") or []      # already an array in Mongo
        ger_by_code[code] = ger

    return ger_by_code

GER_BY_CODE = load_ger_mapping_from_mongo()

# 🔹 Enrich candidate_courses in-place
for c in candidate_courses:
    code = c.get("code")
    if not code:
        continue
    if not c.get("ger"):   # if None or empty, fill from Mongo
        c["ger"] = GER_BY_CODE.get(code, [])


In [49]:
def attach_ger_field(course_doc):
    """
    Ensure course_doc has a 'ger' field by looking it up from GER_MAP.
    If no GER info is found, default to empty list.
    """
    # Use the same key you used when building GER_MAP
    key = course_doc.get("code")   # <-- change if needed (e.g., "course_id")
    if key is None:
        course_doc["ger"] = []
        return course_doc

    course_doc["ger"] = GER_MAP.get(key, [])
    return course_doc


In [50]:
# Map from survey "priorityOrder" labels → internal aspect keys
ASPECT_KEY_MAP = {
    "GER_REQUIREMENTS":   "ger_reqs",
    "PROFESSOR_RATING":   "ratings",
    "MAJOR_REQUIREMENTS": "major_ger",
    "INTERESTS":          "interests",
    "TIME_PREFERENCE":    "time",
}

def weights(rank_order, decay=0.5):
    """
    rank_order: list of aspect keys in priority order (best → worst)
    decay: geometric decay factor between ranks.
    """
    raw = {}
    for rank, aspect in enumerate(rank_order):
        raw[aspect] = decay ** rank

    total = sum(raw.values())
    return {k: v / total for k, v in raw.items()}

def compute_aspect_weights(priority_order, decay=0.5):
    rank_order = [
        ASPECT_KEY_MAP[p]
        for p in priority_order
        if p in ASPECT_KEY_MAP
    ]
    return weights(rank_order, decay=decay)


In [51]:
# Test example
# sample_priority = ["GER_REQUIREMENTS","PROFESSOR_RATING","MAJOR_REQUIREMENTS","INTERESTS","TIME_PREFERENCE"]
# aspect_weights = compute_aspect_weights(sample_priority, decay=0.5)
# aspect_weights


In [52]:
def hhmm_to_min(t):
    if not t or ":" not in t:
        return None
    h, m = t.split(":")
    return int(h) * 60 + int(m)

def score_time(course_doc, time_pref):
    """
    time_pref: ["HH:MM", "HH:MM"] (24h earliest, latest)
    Uses meeting.start_min/end_min from the qualified courses collection.
    """
    mtg = course_doc.get("meeting") or {}
    start = mtg.get("start_min")
    end   = mtg.get("end_min")

    if start is None or end is None:
        return 0.5

    if not time_pref or len(time_pref) < 2:
        return 0.5

    pref_start = hhmm_to_min(time_pref[0])
    pref_end   = hhmm_to_min(time_pref[1])

    if pref_start is None or pref_end is None or pref_start >= pref_end:
        return 0.5

    # Overlap reward
    overlap_start = max(start, pref_start)
    overlap_end   = min(end, pref_end)
    overlap = max(0, overlap_end - overlap_start)
    duration = max(0, end - start)

    if duration > 0 and overlap > 0:
        return 0.7 + 0.3 * (overlap / duration)

    # Otherwise, distance penalty
    if start < pref_start:
        distance = pref_start - start
    else:
        distance = start - pref_end

    SCALE = 120  # 2 hours
    penalty = min(1.0, distance / SCALE)
    return max(0.0, 1.0 - penalty)


In [53]:
def ger_set_builder(course_doc, ger_due, ger_left):
    def extract_tags(lst): 
        tags = set()
        for item in lst or []:
            if isinstance(item, dict):
                tags.update(item.keys())
            elif isinstance(item, str):
                tags.add(item)
        return tags
    
    due_tags = extract_tags(ger_due)
    left_tags = extract_tags(ger_left)
    return due_tags, left_tags, due_tags | left_tags

def score_ger_reqs(course_doc, ger_due, ger_left):
    due_tags, left_tags, _ = ger_set_builder(course_doc, ger_due, ger_left)
    course_tags = set(course_doc.get("ger") or [])
    if not course_tags:
        return 0.0
    if course_tags & due_tags:
        return 1.0
    if course_tags & left_tags:
        return 0.6
    return 0.0



In [54]:
def score_major_ger(course_doc, major_must_set, major_elec_set):
    code = (course_doc.get("code") or "").upper()
    if code in major_must_set:
        return 1.0
    if code in major_elec_set:
        return 1.0
    return 0.05


In [55]:
def score_interests(course_doc, interests):
    if not interests:
        return 0.5

    title = (course_doc.get("title") or "").lower()
    code  = (course_doc.get("code") or "").lower()

    hits = 0
    for raw in interests:
        key = str(raw).lower()
        if key in title or key in code:
            hits += 1

    if hits == 0:
        return 0.2
    return min(1.0, 0.4 + 0.6 * (hits / len(interests)))


import math

def safe_float(x, default=None):
    try:
        if x is None:
            return default
        return float(x)
    except (TypeError, ValueError):
        return default

def score_ratings(course_doc):
    """
    Robust RMP scoring:
    - rating and num_ratings may be strings or missing
    - default rating = 2.9
    - default num_ratings = 0
    """
    rmp = course_doc.get("rmp") or {}
    rating_raw = rmp.get("rating")
    num_raw    = rmp.get("num_ratings")

    rating = safe_float(rating_raw, 2.9)
    num    = safe_float(num_raw, 0.0)

    # Map rating in [1,5] → [0,1]
    base = max(0.0, min(1.0, (rating - 1.0) / 4.0))

    # Popularity bonus based on log(#ratings); if num==0 → 0
    if num <= 0:
        pop = 0.0
    else:
        pop = math.log1p(num) / math.log1p(50)

    return min(1.0, 0.8 * base + 0.2 * pop)



In [56]:
def build_major_sets(major_must, major_elec_groups):
    must_set = set(major_must)
    elec_set = set()
    for group in major_elec_groups:
        for c in group.get("courses", []):
            elec_set.add(c)
    return must_set, elec_set

def build_student_context(shared_id):
    pref = synthetic_prefs[shared_id]
    hist = synthetic_courses[shared_id]

    degree = pref["degreeType"]  # "BA" or "BS"
    major_code = "CSBA" if degree == "BA" else "CSBS"

    year  = pref["year"]                          # "Freshman", ...
    term  = pref["expectedGraduation"]["semester"]  # "Fall"/"Spring"

    major_must, major_elec, ger_due, ger_left = track_grad(
        major_code,
        hist["incoming_test_courses"],
        hist["incoming_transfer_courses"],
        hist["emory_courses"],
        year,
        term,
        countic=True,
    )

    major_must_set, major_elec_set = build_major_sets(major_must, major_elec)

    ctx = {
        "shared_id": shared_id,
        "pref": pref,
        "interests": pref.get("interests", []),
        "timePreference": pref.get("timePreference", []),
        "preferredCredits": pref.get("preferredCredits"),
        "major_must_set": major_must_set,
        "major_elec_set": major_elec_set,
        "ger_due": ger_due,
        "ger_left": ger_left,
    }
    return ctx


In [57]:
def get_required_gers(student_ctx, ignore=("HLTH",)):
    """
    student_ctx['ger_due'] looks like: [{'ECS': 1}, {'HLTH': 1}]
    Return a set of GER codes we must cover at least once,
    excluding anything in `ignore` (e.g. HLTH for Spring).
    """
    ignore = set(ignore or [])
    required = set()
    for d in student_ctx.get("ger_due", []):
        for ger, count in d.items():
            if count > 0 and ger not in ignore:
                required.add(ger)
    return required


In [58]:
def ecs_required(student_ctx):
    """Return True if ECS is in ger_due with count > 0."""
    for d in student_ctx.get("ger_due", []):
        if d.get("ECS", 0) > 0:
            return True
    return False


In [59]:
def _ger_list_to_map(lst):
    """
    Convert [{'ECS': 1}, {'HLTH': 0}, ...] -> {'ECS': 1, 'HLTH': 0, ...}
    """
    out = {}
    for d in lst or []:
        for k, v in d.items():
            out[k] = v
    return out

def get_ger_requirement_maps(student_ctx):
    """
    Use ger_due / ger_left from track_graduation (already stored in student_ctx)
    to decide how many of each GER we should TRY to cover this term.

    For non-seniors -> use ger_due (what must be done by this stage).
    For Senior Spring -> use ger_left (everything still outstanding must be done now).
    """
    ger_due_map  = _ger_list_to_map(student_ctx.get("ger_due"))
    ger_left_map = _ger_list_to_map(student_ctx.get("ger_left"))

    pref      = student_ctx.get("pref", {})
    year_label = pref.get("year")
    grad_info  = pref.get("expectedGraduation", {})

    graduating_this_spring = (
        year_label == "Senior"
        and grad_info.get("semester") == "Spring"
    )

    if graduating_this_spring:
        # Senior Spring: every remaining GER must be finished
        required_ger_counts = {g: c for g, c in ger_left_map.items() if c > 0}
    else:
        # Other years/terms: only enforce those due by this stage
        required_ger_counts = {g: c for g, c in ger_due_map.items() if c > 0}

    return ger_due_map, ger_left_map, required_ger_counts


In [60]:
def get_taken_courses(shared_id):
    """
    Returns a set of course codes the student has already completed,
    based on synthetic_courses[shared_id].
    """
    hist = synthetic_courses[shared_id]  # uses your existing synthetic_courses dict

    incoming_test     = hist.get("incoming_test_courses", []) or []
    incoming_transfer = hist.get("incoming_transfer_courses", []) or []
    emory_courses     = hist.get("emory_courses", []) or []

    taken = set(incoming_test + incoming_transfer + emory_courses)
    return taken

In [61]:
def prereqs_satisfied(course_doc, taken):
    """
    course_doc["prerequisites"] is expected to be List[List[str]]:
      [["MATH111"], ["CS170", "CS171"]]
    interpreted as:
      (MATH111) AND (CS170 OR CS171)

    taken: set of completed course codes for the student.
    """
    prereq_groups = course_doc.get("prerequisites")

    # No prereq info → allow by default
    if not prereq_groups:
        return True
    if not isinstance(prereq_groups, list):
        # If it's some weird type, be permissive
        return True

    for group in prereq_groups:
        if not group:
            # empty group = no condition for this AND term
            continue
        # At least one of the OR-items must be in `taken`
        if not any((code in taken) for code in group):
            return False

    return True

In [62]:
def filter_courses_by_prereqs(candidate_courses, shared_id):
    """
    Given the global candidate_courses list and a student shared_id,
    return ONLY those courses for which:
      - the student has NOT already taken the course, and
      - the student DOES satisfy the prerequisites.
    """
    # Courses the student already has credit for
    taken_raw = get_taken_courses(shared_id)
    taken = {code.upper() for code in taken_raw}

    filtered = []
    for c in candidate_courses:
        code = (c.get("code") or "").upper()

        # 🔹 1) Skip anything the student has already taken
        if code in taken:
            continue

        # 🔹 2) Only keep courses where prereqs are satisfied
        if prereqs_satisfied(c, taken):
            filtered.append(c)

    return filtered


In [63]:
import importlib
import track_graduation
from track_graduation import track_grad  # uses Blue_GER / ger_due / ger_left

def get_ger_status_for_student(shared_id, term="Spring"):
    """
    Returns (major_must, major_elec, ger_due, ger_left) for this student and term,
    using track_graduation.py.
    """
    # Re-load to reset Blue_GER each call (since track_grad mutates it globally)
    importlib.reload(track_graduation)

    student_ctx = build_student_context(shared_id)
    degree_type = student_ctx.get("degreeType", "BA")
    major_code  = "CSBA" if degree_type == "BA" else "CSBS"

    year = student_ctx.get("year", "Freshman")          # "Freshman", "Sophomore", ...
    # For now we assume you're always building Spring schedules
    # If you ever support Fall, pass term dynamically.

    # Grab completed-courses history (however you're storing it now)
    hist = synthetic_courses[shared_id]  # or from Mongo if you moved off synthetic
    incoming_test     = hist.get("incoming_test_courses", [])
    incoming_transfer = hist.get("incoming_transfer_courses", [])
    emory_courses     = hist.get("emory_courses", [])

    major_must, major_elec, ger_due, ger_left = track_grad(
        major_code,
        incoming_test,
        incoming_transfer,
        emory_courses,
        year,
        term,
        countic=True,
    )

    return major_must, major_elec, ger_due, ger_left


In [64]:
def score_course(course_doc, student_ctx, aspect_weights):
    components = {
        "time":       score_time(course_doc, student_ctx["timePreference"]),
        "major_ger":  score_major_ger(course_doc, student_ctx["major_must_set"],
                                      student_ctx["major_elec_set"]),
        "ger_reqs":   score_ger_reqs(course_doc, student_ctx["ger_due"], student_ctx["ger_left"]),
        "ratings":    score_ratings(course_doc),
        "interests":  score_interests(course_doc, student_ctx["interests"]),
    }
    total = 0.0
    for aspect, w in aspect_weights.items():
        total += w * components.get(aspect, 0.0)
    return total, components


In [65]:
def get_ranked_courses(shared_id):
    """
    Returns a DataFrame of *all* candidate courses for this student,
    sorted by total_score descending.
    """
    # Build student context (includes major_must, ger_due, ger_left, prefs, etc.)
    student_ctx = build_student_context(shared_id)
    priority_order = student_ctx["pref"]["priorityOrder"]
    aspect_weights = compute_aspect_weights(priority_order, decay=0.5)

    rows = []

    # Use the global candidate_courses list loaded from COURSES_QUAL_PATH
    # and attach GER tags from Mongo to each one.
    courses_with_ger = [attach_ger_field(c.copy()) for c in candidate_courses]

    for c in courses_with_ger:
        total, comps = score_course(c, student_ctx, aspect_weights)
        meeting = c.get("meeting") or {}
        credits = c.get("credits") or c.get("credit") or 3.0

        rows.append({
            "shared_id": shared_id,
            "code": c.get("code"),
            "title": c.get("title"),
            "ger": c.get("ger") or [],
            "days": meeting.get("days") or [],
            "time_raw": meeting.get("raw"),
            "start_min": meeting.get("start_min"),
            "end_min": meeting.get("end_min"),
            "rmp_rating": (c.get("rmp") or {}).get("rating"),
            "credits": credits,
            "total_score": total,
            "time_score": comps["time"],
            "major_ger_score": comps["major_ger"],
            "ger_reqs_score": comps["ger_reqs"],
            "ratings_score": comps["ratings"],
            "interests_score": comps["interests"],
        })

    df = pd.DataFrame(rows)
    df = df.sort_values("total_score", ascending=False).reset_index(drop=True)
    return df


In [66]:
ranked_000001 = get_ranked_courses("000001")
ranked_000001.head(18)


,shared_id,code,title,ger,days,time_raw,start_min,end_min,rmp_rating,credits,total_score,time_score,major_ger_score,ger_reqs_score,ratings_score,interests_score
0,000001,MUS300R,Ensemble: Orchestra,[XA],"[M, W]",MW 4pm-6pm,960.0,1080.0,5.0,1.0,0.567742,1.0,0.05,0.6,0.80,0.2
1,000001,MUS300R,Ensemble: Orchestra,[XA],[F],F 10am-11:15am,600.0,675.0,5.0,1.0,0.567742,1.0,0.05,0.6,0.80,0.2
2,000001,MUS300R,Ensemble: Orchestra,[XA],[],None,NaN,NaN,5.0,1.0,0.551613,0.5,0.05,0.6,0.80,0.2
3,000001,MUS310R,"Applied Music, Non-Majors: Strings",[XA],[],None,NaN,NaN,5.0,1.0,0.551613,0.5,0.05,0.6,0.80,0.2
4,000001,MUS310R,"Applied Music, Non-Majors: Strings",[XA],[],None,NaN,NaN,5.0,1.0,0.551613,0.5,0.05,0.6,0.80,0.2
5,000001,MUS310R,"Applied Music, Non-Majors: Strings",[XA],[],None,NaN,NaN,5.0,1.0,0.551613,0.5,0.05,0.6,0.80,0.2
6,000001,MUS320R,Applied Music: Majors: Strings,[XA],[],None,NaN,NaN,5.0,2.0,0.551613,0.5,0.05,0.6,0.80,0.2
7,000001,PHYS499R,Undergraduate Research,[XA],[],None,NaN,NaN,5.0,3.0,0.551613,0.5,0.05,0.6,0.80,0.2
8,000001,MUS310R,"Applied Music, Non-Majors: Strings",[XA],[],None,NaN,NaN,5.0,1.0,0.551613,0.5,0.05,0.6,0.80,0.2
9,000001,PSYC499R,Directed Research,[XA],[],None,NaN,NaN,5.0,3.0,0.551613,0.5,0.05,0.6,0.80,0.2


In [67]:
major_must, major_elec, ger_due, ger_left = get_ger_status_for_student("000025")
ger_due, ger_left


MATH112 updated must:
must: ['MATH111', 'MATH221', 'CS170', 'CS171', 'CS224', 'CS253', 'CS255', 'CS326', 'CS350']

CS325 updated group:
choose: 1
courses: ['CS329', 'CS334', 'CS377']

CS452 updated group:
choose: 1
courses: ['CS312', 'CS325', 'CS326', 'CS329', 'CS334', 'CS350', 'CS370', 'CS371W', 'CS377', 'CS385', 'CS424', 'CS441', 'CS443', 'CS444', 'CS463', 'CS470', 'CS480', 'CS485', 'CS495A', 'CS495BW', 'CS497R', 'CS498R']

CS253 updated must:
must: ['MATH111', 'MATH221', 'CS170', 'CS171', 'CS224', 'CS255', 'CS326', 'CS350']

MATH221 updated must:
must: ['MATH111', 'CS170', 'CS171', 'CS224', 'CS255', 'CS326', 'CS350']

CS370 updated group:
choose: 0
courses: ['CS371W']

CS171 updated must:
must: ['MATH111', 'CS170', 'CS224', 'CS255', 'CS326', 'CS350']

CS255 updated must:
must: ['MATH111', 'CS170', 'CS224', 'CS326', 'CS350']

CS312 updated group:
choose: 0
courses: ['CS325', 'CS326', 'CS329', 'CS334', 'CS350', 'CS370', 'CS371W', 'CS377', 'CS385', 'CS424', 'CS441', 'CS443', 'CS444', '

([], [{'XA': 1}])

In [68]:
def blocks_conflict(start1, end1, start2, end2):
    """
    True if two time blocks overlap (strict overlap: start < other_end and other_start < end)
    """
    if start1 is None or end1 is None or start2 is None or end2 is None:
        # If we don't know the time, treat it as "no conflict" (or change to True if you prefer)
        return False
    return (start1 < end2) and (start2 < end1)


In [69]:
def extract_ger_due_tags(ger_due):
    """
    ger_due is usually like: [{"HA": 1}, {"NS": 1}, {"QR": 1}]
    Return a set like {"HA", "NS", "QR"} for tags that are required >= 1.
    """
    tags = set()
    for d in ger_due or []:
        if isinstance(d, dict):
            for tag, count in d.items():
                if count and count > 0:
                    tags.add(tag)
    return tags


def _normalize_days(day_val):
    """
    Convert whatever is in the 'days' field into a set of day tokens.
    Examples:
      "MWF"     -> {"M","W","F"}
      ["M","W"] -> {"M","W"}
      None      -> empty set
    """
    if not day_val:
        return set()
    if isinstance(day_val, str):
        # Treat each character as a day token; adjust this if you use e.g. "Mo"
        return set(day_val)
    try:
        return set(day_val)
    except TypeError:
        return set()

def has_time_conflict(candidate, chosen_courses):
    """
    Given two *course dicts* using start_min/end_min + days, check if they conflict.
    Uses blocks_conflict() which is already defined above.
    """
    cand_days = _normalize_days(candidate.get("days"))
    cand_start = candidate.get("start_min")
    cand_end   = candidate.get("end_min")

    for ch in chosen_courses:
        ch_days = _normalize_days(ch.get("days"))
        # Only care about overlapping days
        if not (cand_days & ch_days):
            continue

        if blocks_conflict(
            cand_start, cand_end,
            ch.get("start_min"), ch.get("end_min")
        ):
            return True

    return False

def pick_required_ger_courses(
    candidates,
    ger_due_tags,
    max_credits,
    conflict_fn,
):
    """
    First-pass greedy selection of courses that satisfy ger_due_tags.
    Each tag tries to get at least one course that carries that GER.
    - candidates: list of course dicts (e.g., from ranked.to_dict('records')).
    - ger_due_tags: set like {"HA","NS"}.
    """
    chosen = []
    used_codes = set()
    total_credits = 0.0

    # Sort once by score
    sorted_cands = sorted(candidates, key=lambda c: c.get("total_score", 0), reverse=True)

    for tag in ger_due_tags:
        # Skip if already covered
        if any(tag in (c.get("ger") or []) for c in chosen):
            continue

        for c in sorted_cands:
            code = c.get("code")
            if code in used_codes:
                continue

            gers = c.get("ger") or []
            if tag not in gers:
                continue

            if conflict_fn(c, chosen):
                continue

            credits = float(c.get("credits", 3.0) or 3.0)
            if max_credits is not None and total_credits + credits > max_credits:
                continue

            chosen.append(c)
            used_codes.add(code)
            total_credits += credits
            break  # done for this tag

    return chosen, used_codes, total_credits

def build_schedule(
    candidates,
    ger_due,
    max_credits,
    n_courses,
    conflict_fn,
):
    """
    Core scheduler:
      1) Ensures each GER in ger_due is represented if at all possible.
      2) Fills remaining slots with the highest-scoring non-conflicting courses.
    Returns:
      - list of chosen course dicts
      - list of unsatisfied GER tags (if no valid course could be found)
    """
    # 1) Identify required GER tags for this term
    ger_due_tags = extract_ger_due_tags(ger_due)

    # 2) First pass: lock in GER-required courses
    chosen, used_codes, total_credits = pick_required_ger_courses(
        candidates=candidates,
        ger_due_tags=ger_due_tags,
        max_credits=max_credits,
        conflict_fn=conflict_fn,
    )

    # 3) Second pass: fill remaining slots
    remaining = [
        c for c in candidates
        if c.get("code") not in used_codes
    ]
    remaining.sort(key=lambda c: c.get("total_score", 0), reverse=True)

    for c in remaining:
        if len(chosen) >= n_courses:
            break

        if conflict_fn(c, chosen):
            continue

        credits = float(c.get("credits", 3.0) or 3.0)
        if max_credits is not None and total_credits + credits > max_credits:
            continue

        chosen.append(c)
        total_credits += credits

    # 4) Track which GER tags we actually satisfied
    satisfied_tags = set()
    for tag in ger_due_tags:
        if any(tag in (c.get("ger") or []) for c in chosen):
            satisfied_tags.add(tag)

    unsatisfied_tags = list(ger_due_tags - satisfied_tags)
    return chosen, unsatisfied_tags


In [70]:
import json
from track_graduation import getridofZ

# load synthetic course history once
with open("synthetic_courses.json") as f:
    _COURSES_BY_ID = {d["shared_id"]: d for d in json.load(f)}

def get_taken_set(shared_id: str) -> set[str]:
    """
    Return a normalized set of course codes the student has *already taken*.
    Uses the same course lists as track_graduation.py.
    """
    data = _COURSES_BY_ID.get(shared_id, {})
    taken = []

    for key in ("incoming_test_courses", "incoming_transfer_courses", "emory_courses"):
        taken.extend(data.get(key, []))

    # normalize like track_graduation (strip Z/ZL, uppercase)
    norm = set()
    for c in taken:
        if not c:
            continue
        c2 = getridofZ(c)
        norm.add(c2.upper())
    return norm


In [71]:
def get_replacement_for_same_ger(shared_id, current_schedule_df, removed_slot_idx):
    ranked = get_ranked_courses(shared_id).copy()
    student_ctx = student_contexts[shared_id]

    # GER(s) we want to preserve – intersect with ger_due if you’d like
    removed_row = current_schedule_df.loc[removed_slot_idx]
    target_gers = removed_row.get("ger", []) or []
    required_gers = get_required_gers(student_ctx)
    target_ger = next((g for g in target_gers if g in required_gers), None)
    if target_ger is None and target_gers:
        target_ger = target_gers[0]

    # other courses fixed in the schedule
    remaining_rows = [
        current_schedule_df.loc[i]
        for i in current_schedule_df.index
        if i != removed_slot_idx
    ]

    used_codes = set(r["code"] for r in remaining_rows)

    def no_time_conflict(row):
        mtg_days = set(row.get("days", []) or [])
        start = row.get("start_min")
        end   = row.get("end_min")
        if start is None or end is None:
            return False
        for r in remaining_rows:
            other_days = set(r.get("days", []) or [])
            if not (mtg_days & other_days):
                continue
            o_start = r.get("start_min")
            o_end   = r.get("end_min")
            if max(start, o_start) < min(end, o_end):
                return False
        return True

    # candidates with the same GER, not already in schedule
    candidates = ranked[
        ranked["ger"].apply(lambda g: target_ger in (g or []))
        & (~ranked["code"].isin(used_codes))
    ]

    for _, row in candidates.iterrows():
        if no_time_conflict(row):
            return row  # top-scoring compatible replacement

    return None  # no valid replacement


In [72]:
import pandas as pd

def build_schedule_for_student(shared_id, n_courses=5):
    """
    Build a schedule that:
      - starts from ranked courses,
      - uses GER counts from track_graduation:
          * for non-seniors: ger_due (things that must be done by this stage)
          * for Senior Spring: ger_left (all remaining GERs must be finished)
      - avoids a schedule dominated by a single GER (like all XA),
      - never repeats a course code,
      - respects credit cap and time conflicts.
    """
    ranked = get_ranked_courses(shared_id).copy()

    # ---- student context & GER requirement maps ----
    student_ctx   = build_student_context(shared_id)
    pref_cred     = float(student_ctx.get("preferredCredits", 15))

    major_must_set = set(student_ctx.get("major_must_set", []))
    major_elec_set = set(student_ctx.get("major_elec_set", []))
    major_all      = major_must_set | major_elec_set

    ger_due_map, ger_left_map, required_ger_counts = get_ger_requirement_maps(student_ctx)

    # how many GERs we are fulfilling THIS term via the schedule
    ger_fulfilled = defaultdict(int)

    schedule_rows = []
    used_codes    = set()
    total_credits = 0.0

    def no_time_conflict(row, current_rows):
        """Return True if row does not overlap any course in current_rows."""
        mtg_days = set(row.get("days", []) or [])
        start    = row.get("start_min")
        end      = row.get("end_min")
        if start is None or end is None:
            # If you want to allow courses with unknown times, change to `return True`
            return False

        for r in current_rows:
            other_days = set(r.get("days", []) or [])
            if not (mtg_days & other_days):
                continue
            o_start = r.get("start_min")
            o_end   = r.get("end_min")
            if max(start, o_start) < min(end, o_end):
                return False
        return True

    def can_add(row, current_rows, cur_credits):
        """Check credits, time, duplicates, and GER caps for non-major courses."""
        code = row["code"]
        if code in used_codes:
            return False

        credits = row.get("credits", 0) or 0
        try:
            credits = float(credits)
        except Exception:
            credits = 0.0

        if cur_credits + credits > pref_cred + 1e-6:
            return False

        if not no_time_conflict(row, current_rows):
            return False

        gers = row.get("ger") or []
        is_major_course = code in major_all

        # --- GER caps only apply to non-major "pure GER" courses ---
        if not is_major_course:
            for g in gers:
                # If this GER is required this term, don't exceed that required count
                if g in required_ger_counts and ger_fulfilled[g] >= required_ger_counts[g]:
                    return False
                # If not required this term, keep it from dominating:
                # allow at most 1 such elective GER in this schedule
                if g not in required_ger_counts and ger_fulfilled[g] >= 1:
                    return False

        return True

    def add_row(row):
        """Add a row to the schedule and update bookkeeping."""
        nonlocal total_credits
        schedule_rows.append(row)
        used_codes.add(row["code"])
        credits = float(row.get("credits", 0) or 0)
        total_credits += credits

        code = row["code"]
        if code not in major_all:
            for g in (row.get("ger") or []):
                ger_fulfilled[g] += 1

    # ---------- 1) Satisfy GERs that are required by this stage ----------
    # required_ger_counts might be, e.g., {'FS': 1, 'PE': 1} for Frosh Spring,
    # or all remaining GERs for Senior Spring.
    for ger, needed in required_ger_counts.items():
        if needed <= 0:
            continue

        ger_candidates = ranked[ranked["ger"].apply(lambda g_list: ger in (g_list or []))]

            # ---------- 2) Fill remaining slots: MAJOR courses first ----------
    # First pass: only CS major courses (must + electives)
    major_ranked = ranked[ranked["code"].isin(major_all)]

    for _, row in major_ranked.iterrows():
        if len(schedule_rows) >= n_courses:
            break
        if not can_add(row, schedule_rows, total_credits):
            continue
        add_row(row)

    # ---------- 3) If there are still open slots, allow non-major electives ----------
    if len(schedule_rows) < n_courses:
        remaining = ranked[~ranked["code"].isin(used_codes)]

        for _, row in remaining.iterrows():
            if len(schedule_rows) >= n_courses:
                break
            if not can_add(row, schedule_rows, total_credits):
                continue
            add_row(row)


    # ---------- 2) Fill remaining slots by best overall score ----------
    for _, row in ranked.iterrows():
        if len(schedule_rows) >= n_courses:
            break
        if not can_add(row, schedule_rows, total_credits):
            continue
        add_row(row)

    # ---------- 3) Return final schedule DataFrame ----------
    if not schedule_rows:
        return pd.DataFrame(columns=["slot", "code", "title", "days",
                                     "time_raw", "credits", "total_score", "ger"])

    sched_df = pd.DataFrame(schedule_rows).reset_index(drop=True)
    sched_df.insert(0, "slot", range(1, len(sched_df) + 1))
    return sched_df


In [73]:
for c in candidate_courses[:10]:
    print(c.get("code"), c.get("ger"))


CL498R []
AAS100 ['ETHN', 'HSC']
MESAS370 ['HSC']
AAS100 ['ETHN', 'HSC']
AAS190 ['ETHN', 'FS']
PHYS116 ['NS']
MATH321 ['MQR']
MESAS354 []
MATH330 ['MQR']
AAS190 ['ETHN', 'FS']


In [74]:
print (build_student_context("000025"))

{'shared_id': '000025', 'pref': {'shared_id': '000025', 'degreeType': 'BA', 'year': 'Junior', 'expectedGraduation': {'semester': 'Fall', 'year': '2027'}, 'preferredCredits': 15, 'interests': ['Software Engineering', 'AI/ML'], 'timePreference': ['10:00', '16:00'], 'timeUnavailable': [], 'priorityOrder': ['GER_REQUIREMENTS', 'PROFESSOR_RATING', 'INTERESTS', 'TIME_PREFERENCE', 'MAJOR_REQUIREMENTS']}, 'interests': ['Software Engineering', 'AI/ML'], 'timePreference': ['10:00', '16:00'], 'preferredCredits': 15, 'major_must_set': set(), 'major_elec_set': set(), 'ger_due': [], 'ger_left': [{'XA': 1}]}


In [75]:
schedule_1 = build_schedule_for_student("000025", n_courses=5)
schedule_1


,slot,shared_id,code,title,ger,days,time_raw,start_min,end_min,rmp_rating,credits,total_score,time_score,major_ger_score,ger_reqs_score,ratings_score,interests_score
0,1,000025,MUS300R,Ensemble: Orchestra,[XA],[F],F 10am-11:15am,600.0,675.0,5.0,1.0,0.608065,1.0,0.05,0.6,0.800000,0.2
1,2,000025,GRK370,Spec Topics: Greek Literature,[],"[M, W, F]",MWF 11:30am-12:20pm,690.0,740.0,4.5,3.0,0.310522,1.0,0.05,0.0,0.847024,0.2
2,3,000025,OAM432,Negotiations,[],"[T, Th]",TTh 1pm-2:15pm,780.0,855.0,5.0,3.0,0.298387,1.0,0.05,0.0,0.800000,0.2
3,4,000025,MUS212,Exploring Choral Music,[HA],"[M, W]",MW 10am-11:15am,600.0,675.0,5.0,3.0,0.298387,1.0,0.05,0.0,0.800000,0.2
4,5,000025,THEA120,Acting: Fundamentals,[],"[T, Th]",TTh 11:30am-12:45pm,690.0,765.0,5.0,3.0,0.298387,1.0,0.05,0.0,0.800000,0.2


In [76]:
student_ctx = build_student_context("000025")
get_required_gers(student_ctx)
ranked_000001 = get_ranked_courses("000025")
ranked_000001[ranked_000001["ger"].apply(lambda g: "ECS" in (g or []))].head()
ranked_000001[ranked_000001["ger"].apply(lambda g: "HLTH" in (g or []))].head()

,shared_id,code,title,ger,days,time_raw,start_min,end_min,rmp_rating,credits,total_score,time_score,major_ger_score,ger_reqs_score,ratings_score,interests_score
